In [1]:
import torch
import csv
import glob
from torchdrug import data as da
from torchdrug.core import Registry as R
from rdkit import Chem
from torchdrug import transforms
import pickle
import math
import pdbfixer
import openmm.app as app
import pandas as pd
import numpy as np
import os
from torchdrug import layers
from torchdrug.layers import geometry
from torchdrug import models

In [2]:
def translate_mutation(raw_mutant):
    list_mutants = raw_mutant.split(":")
    muts=[]
    for raw_mut in list_mutants:
        re = raw_mut[:1]
        to = raw_mut[-1]
        pos = int(raw_mut[1:-1])
        mut=three_letter[re]+'-'+str(pos)+'-'+three_letter[to]
        muts.append(mut)
    return muts

def get_mut(tsv_path):
    mm1=[]
    mm01=[]    
    
    table = pd.read_csv(tsv_path)    
    for raw_mutant in table['mutant']:
        mm1=translate_mutation(raw_mutant)
        mm01.append(mm1)    
    x=np.array(mm01,dtype=object)
    return table, x

def get_pdb(table, x, dir_name):
    for i in range (len(x)):
        fixer = pdbfixer.PDBFixer("./data/VKOR1_HUMAN.pdb")
        for j in range (len(x[i])):
            fixer.applyMutations([x[i][j]], "A")
    #    fixer.findMissingResidues()
    #    fixer.findMissingAtoms()
    #    fixer.addMissingAtoms()
    #    fixer.addMissingHydrogens(7.0)
    #    file_name=str(i)+'_'+np.array(mutations)[i]+'.pdb'
        file_name=table['num'][i].astype(str)+'.pdb'
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        os.chdir(dir_name)
        with open(file_name, 'w') as f:
            app.PDBFile.writeFile(fixer.topology, fixer.positions,f)
        os.chdir('..')
        os.chdir('..')

def get_graph():
    path = glob.glob('./data/PDB/*.pdb')
    path.sort(key=lambda x:int(x[11:-4]))
    num = len(path)
    print(path[len(path)-1])
    # protein
    proteins=[]
    for pdb_file in path:
        protein = da.Protein.from_pdb(pdb_file, atom_feature="position", bond_feature="length", residue_feature="symbol")
        proteins.append(protein)
    protein_ = da.Protein.pack(proteins)
    protein = graph_construction_model(protein_)
    return num, protein

def get_reps(protein):
    gearnet_edge.eval()
    outputs=[]
    for pro in protein:
        pro = da.Protein.pack([pro])
        output = gearnet_edge(graph=pro, input=pro.residue_feature.float())
        o1 = output['graph_feature']
        for i in range (3072):
            a = o1[0][i].item()
            outputs.append(a)
    arr=np.array(outputs,dtype=np.float32)
    arr_2d = arr.reshape((arr.shape[0]//3072, 3072))
    df = pd.DataFrame(arr_2d)
    return df

graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()], 
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)],
                                                    edge_feature="gearnet")
gearnet_edge = models.GearNet(input_dim=21, hidden_dims=[512, 512, 512, 512, 512, 512],
                              num_relation=7, edge_input_dim=59, num_angle_bin=8,
                              batch_norm=True, concat_hidden=True, short_cut=True, readout="sum")

one_letter ={'VAL':'V', 'ILE':'I', 'LEU':'L', 'GLU':'E', 'GLN':'Q', 
             'ASP':'D', 'ASN':'N', 'HIS':'H', 'TRP':'W', 'PHE':'F', 'TYR':'Y',
             'ARG':'R', 'LYS':'K', 'SER':'S', 'THR':'T', 'MET':'M', 'ALA':'A',
             'GLY':'G', 'PRO':'P', 'CYS':'C'}
three_letter ={'V':'VAL', 'I':'ILE', 'L':'LEU', 'E':'GLU', 'Q':'GLN',
               'D':'ASP', 'N':'ASN', 'H':'HIS', 'W':'TRP', 'F':'PHE', 'Y':'TYR',
               'R':'ARG', 'K':'LYS', 'S':'SER', 'T':'THR', 'M':'MET', 'A':'ALA',
               'G':'GLY', 'P':'PRO', 'C':'CYS'}

In [3]:
dir_name = './data/PDB'
tsv_path = './data/VKOR1_HUMAN_Chiasson_abundance_2020.csv'
table, mut = get_mut(tsv_path)
get_pdb(table, mut, dir_name)

pthfile = 'mc_gearnet_edge.pth'
#pthfile = 'attr_gearnet_edge.pth'
#pthfile = 'angle_gearnet_edge.pth'
#pthfile = 'dihedral_gearnet_edge.pth'
#pthfile = 'distance_gearnet_edge.pth'

net = torch.load(pthfile)
gearnet_edge.load_state_dict(net)

num, protein = get_graph()
df = get_reps(protein)
df.to_csv('VKOR1-mc.csv')

./data/PDB/2695.pdb
